In [1]:
import numpy as np
import pretty_midi
import glob
import os
import random
from tqdm import tqdm

In [ ]:
def convert_midi(path, time_step):
    """
    convert midi data to vector
    
    params
    ---------
    path: list
        midi data path
    time_step: int
        time step of note
    
    returns
    ---------
    midi_vector: list
        converted midi
    """
    #initialize
    midi = [] #midi data
    tempo = np.zeros(len(path)) #tempo
    midi_vector = [] #converted midi
    
    #load midi data
    for i in tqdm(range(len(path)), "data input"):
        midi.append(pretty_midi.PrettyMIDI(path[i]))
        tempo_tmp = midi[i].get_tempo_changes()[1]
        if len(tempo_tmp) > 0:
            tempo[i] = tempo_tmp[0]
        else:
            tempo[i] = tempo_tmp
        #round tempo
        if tempo[i] - int(tempo[i]) >= 0.5:
            tempo[i] = int(tempo[i] + 1)
        else:
            tempo[i] = int(tempo[i])
                
    #convert midi data
    for i in tqdm(range(len(path)), desc='convert midi'):
        split = 60/(tempo[i]*time_step/4)
        length = []
        instrument_number = len(midi[i].instruments)
        for n in range(instrument_number):
            melody_tmps = midi[i].instruments[n].notes
            vector_length = int(melody_tmps[-1].end/split) +1
            length.append(vector_length)
        vector_length = max(length)
        midi_vector_tmp = np.zeros((3, vector_length, 128)) #data of 1 song
        for n in range(instrument_number):
            melody_tmps = midi[i].instruments[n].notes
            for melody_tmp in melody_tmps:
                start = int(melody_tmp.start/split)
                end = int(melody_tmp.end/split)
                pitch = melody_tmp.pitch  
                #split time step
                for k in range(start, end):
                    midi_vector_tmp[0][k][pitch] = 70
                    if k == start:
                        midi_vector_tmp[1][k][pitch] = 70
        midi_vector_tmp = (midi_vector_tmp/63.5)-1
        midi_vector.append(midi_vector_tmp)
    
    return midi_vector
    

In [ ]:
def split_bar(data, split_number, texts):
    """
    split midi data per bar

    params
    ---------
    data: list 
        midi
    split_number: int 
        split number of bar
    texts: list 
        text data

    returns
    ---------
    split_data: list 
    splited_texts: list
    """
    split_data = []
    splited_texts = []
    for d_idx, d in enumerate(data):
        length = d.shape[1]
        num_data = int(length/split_number)
        tmp = d.transpose(1, 0, 2)
        for i in range(num_data):
            split_data.append(tmp[i*split_number:(i+1)*split_number].transpose(1, 0, 2))
            splited_texts.append(texts[d_idx])
    
    return split_data, splited_texts

In [ ]:
def save_data(data, texts, dict_path):
    #save data
    for i in tqdm(range(len(data)) ,desc=f"save to {dict_path}"):
        save_dict = {"midi": data[i], "text": texts[i]}
        number = str(i).zfill(len(str(len(data)))+1)
        np.save(f"{dict_path}/{number}.npy", save_dict)

In [ ]:
#get midi data path
directory = "midi/jazz"
path = sorted(glob.glob(f"{directory}/*/*.mid*"))

#get text data
use_text = True
text_path = "data/labels/cultural_background/jazz.txt"
texts = []
if use_text:
    with open(text_path) as f:
        texts = f.readlines()
        texts = [t.replace("\n", "") for t in texts]
else:
    texts = [None]*len(path)
    
#params
time_step = 128 #split time step of note
bar = 8 #split number of bar
split_number = time_step*bar

#split train, validation and test data
train_path = path[:int(len(path)*0.8)]
train_texts = texts[:int(len(path)*0.8)]
val_path = path[int(len(path)*0.8):-5]
val_texts = texts[int(len(path)*0.8):-5]
test_path = path[-5:]
test_text = texts[-5:]

In [ ]:
#load midi data
midi_train = convert_midi(train_path, time_step)
midi_val = convert_midi(val_path, time_step)
midi_test = convert_midi(test_path, time_step)

#split midi data per bar
midi_train, train_texts_splited = split_bar(midi_train, split_number, train_texts)
midi_val, val_texts_splited = split_bar(midi_val, split_number, val_texts)
midi_test, test_texts_splited = split_bar(midi_test, split_number, test_text)

print(f"総データ数{len(midi_train)+len(midi_val)+len(midi_test)}")
print(f"trainデータ数{len(midi_train)}, valデータ数{len(midi_val)}, testデータ数{len(midi_test)}")

convert midi: 100%|██████████| 5/5 [00:00<00:00, 24.19it/s]

総データ数1229
trainデータ数1003, valデータ数152, testデータ数74


In [ ]:
#get save path
folder_name = "data/npy/jazz_1024_background_labeled" 
train_dir = os.path.join(folder_name, "train")
os.makedirs(train_dir, exist_ok=True)
val_dir = os.path.join(folder_name, "val")
os.makedirs(val_dir, exist_ok=True)
test_dir = os.path.join(folder_name, "test")
os.makedirs(test_dir, exist_ok=True)

#save data
save_data(midi_train, train_texts_splited, train_dir)
save_data(midi_val, val_texts_splited, val_dir)
save_data(midi_test, test_texts_splited, test_dir)

save to data/npy/jazz_1024_background_labeled/train: 100%|██████████| 1003/1003 [00:06<00:00, 144.95it/s]
save to data/npy/jazz_1024_background_labeled/val: 100%|██████████| 152/152 [00:01<00:00, 144.11it/s]
save to data/npy/jazz_1024_background_labeled/test: 100%|██████████| 74/74 [00:00<00:00, 143.81it/s]
